In [ ]:
# Erklärung: Importiert YOLO, lädt ein vortrainiertes Modell, sucht die Klassen-ID des Zielobjekts,
# und führt Inferenz aus. Der Code ist hier auskommentiert, damit die Zelle nicht ausgeführt wird.
# from ultralytics import YOLO
#
#
#
# # Load the YOLOv8 pre-trained model
# model = YOLO('yolov8n.pt')  # nano model for quick inference
#
#
# # Get the class ID for the target object
# target_object_name = "cup"  # Replace with your target object name
# class_names = model.names
# target_class_id = next((class_id for class_id, name in class_names.items() if name == target_object_name), None)
#
#
#
# if target_class_id is not None:
#     # Perform inference
#     results = model('images/table_scene.jpeg', save = True)  # Replace with your image path
#
#     # Filter bounding boxes for the target object
#     detections = results[0].boxes
#     specific_boxes = [
#         box.xyxy[0].tolist()
#         for box in detections
#         if int(box.cls[0]) == target_class_id
#     ]
#
#     print(f"Bounding boxes for '{target_object_name}': {specific_boxes}")
# else:
#     print(f"Object name '{target_object_name}' not found in the model's class names.")

### Utils 


In [ ]:
# Erklärung: Hilfsfunktionen zum Zeichnen von Bounding-Boxes auf Bildern und zum Parsen von textuellen Box-Antworten.
# Die Implementierung ist auskommentiert, damit die Zelle nicht ausgeführt wird.
#
# import json
# import re
# from PIL import Image, ImageDraw
# from PIL import ImageColor
#
# additional_colors = [colorname for (colorname, colorcode) in ImageColor.colormap.items()]
#
# #this function is needed to plot bounding boxes on images 
# def plot_bounding_boxes(im, positions):
#     """
#     Plots bounding boxes on an image with markers for each noun phrase, using PIL, normalized coordinates, and different colors.
#
#     Args:
#         img_path: The path to the image file.
#         noun_phrases_and_positions: A list of tuples containing the noun phrases
#          and their positions in normalized [y1 x1 y2 x2] format.
#     """
#
#     # Load the image
#     img = im
#     width, height = img.size
#     print(img.size)
#     # Create a drawing object
#     draw = ImageDraw.Draw(img)
#
#     # Define a list of colors
#     colors = [
#     'red',
#     'green',
#     'blue',
#     'yellow',
#     'orange',
#     ] + additional_colors
#
#     # Iterate over the noun phrases and their positions
#     for i, ((y1, x1, y2, x2)) in enumerate(positions):
#         # Select a color from the list
#         color = colors[i % len(colors)]
#
#         # Convert normalized coordinates to absolute coordinates
#         abs_x1 = int(x1/1000 * width)
#         abs_y1 = int(y1/1000 * height)
#         abs_x2 = int(x2/1000 * width)
#         abs_y2 = int(y2/1000 * height)
#
#         # Draw the bounding box
#         draw.rectangle(
#             ((abs_x1, abs_y1), (abs_x2, abs_y2)), outline=color, width=4
#         )
#
#         # Draw the text
#         #draw.text((abs_x1 + 8, abs_y1 + 6), noun_phrase, fill=color)
#
#     # Display the image
#     img.show()
#
# # if the boxes coordinates are output not as json but as text, should be parsed first
# def parse_list_boxes(text):
#   result = []
#   for line in text.strip().splitlines():
#     # Extract the numbers from the line, remove brackets and split by comma
#     try:
#       numbers = line.split('[')[1].split(']')[0].split(',')
#     except:
#       numbers =  line.split('- ')[1].split(',')
#
#     # Convert the numbers to integers and append to the result
#     result.append([int(num.strip()) for num in numbers])
#
#   return result
#
#
#
#

# VLM (1): OPEN-AI

In [ ]:
# Erklärung: Setup für OpenAI-Client, Bild-Encoder-Funktion und Bildpfad. Code auskommentiert.
# import openai
# from dotenv import load_dotenv  
# import base64
#
# # Function to encode the image
# def encode_image(image_path):
#   with open(image_path, "rb") as image_file:
#     return base64.b64encode(image_file.read()).decode('utf-8')
#
# load_dotenv()
# openAIclient = openai.OpenAI()
#
# img = "images/table_scene.jpeg"
#

In [ ]:
# Erklärung: Beispielaufruf an GPT4 mit Bild, um Koordinaten einer Tasse im JSON-Format zurückzubekommen.
# Der eigentliche Aufruf ist auskommentiert, damit beim Laden keine API-Aufrufe erfolgen.
# import json
#
# completion = openAIclient.chat.completions.create(
#     model="gpt-4.1-mini",
#     messages=[
#         {
#             "role": "user",
#             "content": [
#                 {"type": "text", "text": "Detect if there is a cup in the image and reutrn its coordinates as a list in the format [ymin,xmin, ymax, xmax], normalize the coordinate to 0-1000. Just output the list in json."},
#                 {
#                     "type": "image_url",
#                     "image_url": {
#                         "url": f"data:image/jpeg;base64,{encode_image(img)}",
#                     }
#                 },
#             ],
#         }
#     ],
#     response_format={"type": "json_object"},
# )
#
# # Wrap the text to a specified width
# response = str(completion.choices[0].message.content)
#
#

Load answer in a python dict and/or parse the json before if needed

In [ ]:
# Erklärung: Lade die JSON-Antwort in ein Python-Dictionary (auskommentiert, damit kein Fehler auftritt falls `response` fehlt).
# detection = json.loads(response)

In [ ]:
# Erklärung: Zeige das geladene Detection-Objekt an (auskommentiert).
# detection

now parse and plot bounding boxes. Consider the format the utils function want the bounding boxes to be plotted!

In [ ]:
# Erklärung: Bereitet die Box-Koordinaten vor und ruft die Hilfsfunktion zum Plotten auf.
# Die Zeilen sind auskommentiert, so dass nichts ausgeführt wird beim Öffnen.
# boxes= parse_list_boxes(str(detection['coordinates'])) #depending whether you managed to output a json or not. If your output is still in "str" form you might need to parse it with this util function
# boxes= [detection['coordinates']]
# print(boxes)
# plot_bounding_boxes(Image.open(img), positions=list(boxes))

# VLM (2): GEMINI


In [ ]:
# Erklärung: Setup für Google Gemini-Client (API-Key aus der Umgebung). Code ist auskommentiert.
# %matplotlib inline
# from dotenv import load_dotenv  
# from google import genai
# from PIL import Image
#
# load_dotenv()
# client = genai.Client()
#

In [ ]:
# Erklärung: Anfrage an Gemini senden, um die Position der Tasse zu bestimmen und die Antwort als JSON auszugeben.
# Der eigentliche Aufruf ist auskommentiert, damit keine API-Anfrage ausgeführt wird.
# im = Image.open(img)
#
# response = client.models.generate_content(
#     model="gemini-2.5-flash",
#     contents=[
#         im,
#         (
#             "Detect if there is a cup in the image and reutrn its coordinates as a list in the format [ymin,xmin, ymax, xmax], normalize the coordinate to 0-1000. answer should be json format only"
#         ),
#         
#     ],
#     config={"response_mime_type": "application/json"},
# )
#
# print(response.text)
#

Ah! Output is a list! because I asked for it in the prompt - either you leave it and take account in parsing, or you remove the word "list" in prompt.

In [ ]:
# Erklärung: Versucht, die JSON-Antwort von Gemini zu laden (auskommentiert).
# json.loads(response.text)

In [ ]:
# Erklärung: Parsen der Antwort und Plotten der Bounding-Box (auskommentiert).
# boxes= parse_list_boxes(response.text) # same comment as above
# box_coord = [json.loads(response.text)[0]['box_2d']] #due to the fact it is a list, otherwise this line might change
# plot_bounding_boxes(im,positions=list(box_coord))